<a href="https://colab.research.google.com/github/30-na/COVID-19_community_transmission_infection_prevalence/blob/main/load_rt_large_centeral_metro_counties_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
remotes::install_github("epiforecasts/EpiNow2", dependencies = TRUE)


vctrs        (0.6.0  -> 0.6.1    ) [CRAN]
tibble       (3.2.0  -> 3.2.1    ) [CRAN]
pillar       (1.8.1  -> 1.9.0    ) [CRAN]
cli          (3.6.0  -> 3.6.1    ) [CRAN]
ps           (1.7.2  -> 1.7.3    ) [CRAN]
digest       (0.6.30 -> 0.6.31   ) [CRAN]
Rcpp         (NA     -> 1.0.10   ) [CRAN]
htmltools    (0.5.4  -> 0.5.5    ) [CRAN]
jsonlite     (1.8.3  -> 1.8.4    ) [CRAN]
R.utils      (NA     -> 2.12.2   ) [CRAN]
R.oo         (NA     -> 1.25.0   ) [CRAN]
R.methodsS3  (NA     -> 1.8.2    ) [CRAN]
highr        (0.9    -> 0.10     ) [CRAN]
stringr      (1.4.1  -> 1.5.0    ) [CRAN]
bslib        (0.4.1  -> 0.4.2    ) [CRAN]
lazyeval     (NA     -> 0.2.2    ) [CRAN]
matrixStats  (NA     -> 0.63.0   ) [CRAN]
checkmate    (NA     -> 2.1.0    ) [CRAN]
gtable       (0.3.2  -> 0.3.3    ) [CRAN]
RcppEigen    (NA     -> 0.3.3.9.3) [CRAN]
RcppParallel (NA     -> 5.1.7    ) [CRAN]
isoband      (0.2.6  -> 0.2.7    ) [CRAN]
parallelly   (NA     -> 1.35.0   ) [CRAN]
listenv      (NA     -> 0.9.0    )

Installing 54 packages: vctrs, tibble, pillar, cli, ps, digest, Rcpp, htmltools, jsonlite, R.utils, R.oo, R.methodsS3, highr, stringr, bslib, lazyeval, matrixStats, checkmate, gtable, RcppEigen, RcppParallel, isoband, parallelly, listenv, globals, formatR, dplyr, pkgload, R.cache, hunspell, here, webshot, rex, BH, loo, gridExtra, inline, StanHeaders, future, futile.options, lambda.r, styler, spelling, precommit, kableExtra, covr, rstan, truncnorm, runner, rstantools, progressr, patchwork, future.apply, futile.logger

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/Rtmp31Cj0J/remotes7e312cedda/epiforecasts-EpiNow2-f4b836e/DESCRIPTION’ ... OK
* preparing ‘EpiNow2’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘EpiNow2_1.3.4.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [20]:
# Connect to GitHub
repo_url <- "https://github.com/30-na/COVID-19_community_transmission_infection_prevalence"
dest_dir <- "./COVID-19_community_transmission_infection_prevalence"

if (file.exists(dest_dir)) {
  cat("The repository already exists. Updating with git pull...\n")
  setwd(dest_dir)
  system("git pull")
  setwd("..")
} else {
  cat(paste0("The repository doesn't exist. Cloning from ", repo_url, "...\n"))
  system(paste0("git clone ", repo_url, " ", dest_dir))
  setwd("..")
}


ERROR: ignored

In [5]:
# load library
library(dplyr)
library(EpiNow2)
library(data.table)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”

Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




In [21]:
# load NewCase data county level

load("counties.transmission.newCase.RDA")
head(counties.transmission.newCase)

state,fips_code,county_name,date,confirm,UR_code,UR_category,community_transmission_level
<chr>,<dbl>,<chr>,<date>,<dbl>,<dbl>,<fct>,<chr>
AL,1005,Barbour County,2022-10-17,0.00000,6,Noncore,low
AL,1041,Crenshaw County,2022-10-17,5.00000,6,Noncore,moderate
AL,1059,Franklin County,2022-10-17,20.42840,6,Noncore,substantial
AL,1061,Geneva County,2022-10-17,13.16472,4,Small metro,moderate
AL,1099,Monroe County,2022-10-17,5.00000,6,Noncore,moderate
AL,1119,Sumter County,2022-10-17,11.04869,6,Noncore,substantial


# New Section

In [8]:
# 1. define Targets
target.NCHS = "Large central metro"
names(counties.transmission.newCase)

[1] "state"                        "fips_code"                   
[3] "county_name"                  "date"                        
[5] "confirm"                      "UR_code"                     
[7] "UR_category"                  "community_transmission_level"

In [9]:
# set number of cores to use fitting the model
options(mc.cores = 4)
# no benefit on runtime if cores > chains which is set to 4 by default

# literature distributions 
generation_time <- get_generation_time(disease = "SARS-CoV-2", source = "ganyani", fixed = TRUE)
incubation_period <- get_incubation_period(disease = "SARS-CoV-2", source = "lauer", fixed = TRUE)

# define reporting delay as lognormal with mean of 4 days and sd of 1 day in absence of
# evidence. If data on onset -> report then can use estimate_delay to estimate the delay
reporting_delay <- list(mean = convert_to_logmean(4, 1),
                        mean_sd = 0.1,
                        sd = convert_to_logsd(4, 1),
                        sd_sd = 0.1,
                        max = 15)

In [10]:
# filter list of Large Centeral Metro Counties
LCM_counties = counties.transmission.newCase %>%
filter(
UR_category == target.NCHS
) %>%
  mutate(
  state_fips = paste(state, fips_code, county_name, sep="_")
  )
  names(LCM_counties)

[1] "state"                        "fips_code"                   
[3] "county_name"                  "date"                        
[5] "confirm"                      "UR_code"                     
[7] "UR_category"                  "community_transmission_level"
[9] "state_fips"

In [22]:
# list of target counties 
target.counties.list = unique(LCM_counties$state_fips)

# initial dataframe
#original.columns = c("date", "rt.mean", "county")
#LCM_counties_rt = data.frame(matrix(nrow = 0, ncol = length(original.columns)))   
#colnames(LCM_counties_rt) = LCM_counties_rt


for(i in 1:length(target.counties.list)){
  # set target county
  target.county = target.counties.list[i]
  
  # set confirm case for the target county
  reported_cases = LCM_counties %>%
  dplyr::filter(
    state_fips == target.county
  ) %>%
    dplyr::mutate(
    confirm = as.integer(confirm)
    ) %>%
  dplyr::select(
    date,
    confirm
  )%>%
  dplyr::arrange(
    date
    )
    
  # run epinow
out <- epinow(reported_cases = reported_cases, 
              generation_time = generation_time,
              delays = delay_opts(incubation_period, reporting_delay),
              rt = rt_opts(prior = list(mean = 1.5, sd = 0.5)),
              # here we define the quality of the gaussian process approximation
              # if the fit to data appears poor try increasing basis_prop and
              # potentially the boundary_scale (see ?gp_opts for details)
              # though this will likely increase runtimes.
              gp = gp_opts(basis_prop = 0.2),
              # in some instances stan chains can get stuck when estimating in 
              # these instances consider using the future fitting mode by passing 
              # `future = TRUE, max_execution_time = 60 * 30` to stan_opts and calling 
              # `future::plan("multiprocess")` prior to running epinow this will time out
              # chains after 30 minutes but still return results from completed chains
              stan = stan_opts(),
              horizon = 14, 
              #target_folder = "results",
              #logs = file.path("logs", Sys.Date()),
              #return_output = TRUE, 
              verbose = TRUE)

# make a data frame
r = (out[["estimates"]][["summarised"]]$variable) == "R"
rt.mean = out[["estimates"]][["summarised"]]$mean[r]
date = out[["estimates"]][["summarised"]]$date[r]

rt.df = data.frame(
  date = as.Date(date, "%Y-%m-%d"),
  rt.mean = rt.mean,
  county = target.counties.list[i]
)

# merged all data
#LCM_counties_rt = rbind(
#    LCM_counties_rt,
#    rt.df
#  )

# save file to github
write.csv(rt.df,
file = paste(target.counties.list[i], "rt.df.csv", sep="_"))
  
}


Logging threshold set at INFO for the EpiNow2 logger

Writing EpiNow2 logs to the console and: /tmp/Rtmp31Cj0J/regional-epinow/2022-10-17.log

Logging threshold set at INFO for the EpiNow2.epinow logger

Writing EpiNow2.epinow logs to the console and: /tmp/Rtmp31Cj0J/epinow/2022-10-17.log



DEBUG [2023-03-23 15:19:15] epinow: Running in exact mode for 2000 samples (across 4 chains each with a warm up of 250 iterations each) and 460 time steps of which 14 are a forecast
WARN [2023-03-23 15:56:41] epinow: There were 2000 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them. - 
WARN [2023-03-23 15:56:41] epinow: Examine the pairs() plot to diagnose sampling problems
 - 
WARN [2023-03-23 15:56:46] epinow: The largest R-hat is 3.4, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat - 
WARN [2023-03-23 15:56:59] epinow: Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess - 
WARN [2023-03-23 15:57:13] epinow: Tail Effective Sample

ERROR: ignored

In [1]:
library(ggplot2)
ggplot(LCM_counties_rt,
aes(x=date, y=rt.mean, col=county)) +
  geom_line()

ERROR: ignored

In [23]:

write.csv(rt.df,
file = paste(target.counties.list[i], "rt.df.csv", sep="_"))
  